In [1]:
import os
import chainlit as cl
from langchain import PromptTemplate
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [2]:
DATA_PATH = 'knowledge/docs/'
DB_FAISS_PATH = 'knowledge/vectorstore/db_faiss'

In [3]:
def create_vector_db():
    if os.path.exists(DB_FAISS_PATH):
        print('FAISS DB already exists')

    else:
        loader = DirectoryLoader(
                                DATA_PATH,
                                glob='*.pdf',
                                loader_cls=PyPDFLoader
                                )

        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                    chunk_overlap=50)
        texts = text_splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(
                                            model_name='sentence-transformers/all-MiniLM-L6-v2',
                                            model_kwargs={'device': 'mps'}
                                            )

        db = FAISS.from_documents(texts, embeddings)
        db.save_local(DB_FAISS_PATH)

        print('FAISS DB created')

create_vector_db()

FAISS DB already exists


In [4]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(
                            template=custom_prompt_template,
                            input_variables=['context', 'question']
                            )
    return prompt

def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

def load_llm():
    llm = CTransformers(
                        model = "TheBloke/Llama-2-7B-Chat-GGML",
                        model_type="llama",
                        max_new_tokens = 512,
                        temperature = 0.5
                        )
    return llm

def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'mps'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [5]:
final_result('What is the difference between a cold and the flu?')

2023-09-27 10:04:49 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/1zuu/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


2023-09-27 10:04:52 - Loading faiss.
2023-09-27 10:04:52 - Successfully loaded faiss.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'What is the difference between a cold and the flu?',
 'result': 'The main difference between a cold and the flu is that the flu is caused by an actual virus, whereas a cold is usually caused by other factors such as viruses, bacteria, or environmental irritants.',
 'source_documents': [Document(page_content='Although acute and chronic bronchitis are both\ninflammations of the air passages, their causes and treat-ments are different. Acute bronchitis is most prevalent inwinter. It usually follows a viral infection, such as a coldor the flu, and can be accompanied by a secondary bacter-ial infection. Acute bronchitis resolves within two weeks,although the cough may persist longer. Acute bronchitis,\nlike any upper airway inflammatory process, can increasea person’s likelihood of developing pneumonia .', metadata={'source': 'knowledge/docs/knowledge.pdf', 'page': 611}),
  Document(page_content='Like all other antibiotics, aminoglycosides are not\neffective against influenza , t